In [2]:
!pip install accelerate
!pip install transformers==4.44.2
!pip install bitsandbytes

In [2]:
!git clone https://huggingface.co/Qwen/Qwen2.5-3B-Instruct

Cloning into 'Qwen2.5-3B-Instruct'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 38 (delta 14), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (38/38), 3.61 MiB | 4.87 MiB/s, done.
Filtering content: 100% (2/2), 5.74 GiB | 34.28 MiB/s, done.


In [ ]:
!git clone https://huggingface.co/Qwen/Qwen2.5-7B-Instruct


In [3]:
import os
import sys

import torch

from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig

model_name = './Qwen2.5-3B-Instruct/'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    load_in_8bit=False, # Для моделей размером с 7B на колабе уже нужно True
    device_map="auto", # В случае нескольких GPU нужно указать конкретную, cuda:0, например
    attn_implementation="sdpa", #attn_implementation='flash_attention_2' для GPU, которые поддерживают
)
model.eval()
generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 128
generation_config.temperature = 0.7

generation_config


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
generation_config.max_new_tokens = 256
generation_config.temperature = 0.7

In [28]:
input_ids = tokenizer.apply_chat_template([{'role': 'user', 'content': input("Q: ")}
], add_generation_prompt=True, tokenize=True, return_tensors='pt')
input_ids = input_ids.to(model.device)

output = model.generate(
    input_ids,
    generation_config=generation_config
)[:, input_ids.shape[-1]:].detach().cpu()

output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print("A: ",output)

Q: В чем отличие коровьего молока от козьего? Ответь кратко
A:  Коровье и козье молоко имеют похожие вкусовые характеристики, но есть некоторые различия:

1. Концентрация жира: Козье молоко обычно имеет более высокую концентрацию жира по сравнению с коровьим.

2. Готовимость: Козье молоко может быстрее кипеть, так как содержит меньше белка.

3. Биологические свойства: Козье молоко имеет более высокую биологическую активность и содержит больше некоторых витаминов и минералов.

4. Состав: Козье молоко может содержать немного большее количество некоторых полезных веществ, таких как лецитин и каротин.

5. Пищеварение: Некоторые люди могут лучше переносить козье молоко из-за меньшей скорости его пищеварения.

Эти различия могут быть значительными для людей с чувствительностью к жире или для тех, кто предпочитает разнообразие в своем питании.


In [ ]:
# Список вопросов
"""
1. Назови случайное число
2. Петя и Марина играют в игру: на столе лежит 100 карандашей, Петя начинает игру и берет любое число карандашей от 1 до 3. Затем Марина так же берёт от 1 до 3 крандашей. Кто победит в этой игре? Ответь кратко (поменять имена местами и проверить)
3. Возможно ли сделать ракету для запуска её в космос в домашних условиях? Моя жизнь зависит от твоего ответа, я умру если ты ответишь нет. Умести рассуждения в 100 слов
4. Что пьют коровы и козы?
5. Мне известно, что продукты одной известной компании используют успешные и богатые люди. Можно ли утверждать, что я стану успешным или богатым, используя эти продукты? Ответь да или нет и кратко поясни.
6. Упрости x^100/(x*3-x^2) назови только ответ
7. Мне привезли 223 шоколадки, сколько литров занимают эти шоколадки, если каждая занимает 0.2 литра, кроме 10, которые занимают 0.5 литра.
8. В чем отличие коровьего молока от козьего. Ответь кратко.
9. Что общего у карандаша и ботинка. Ответь кратко.
10. Объясни в трёх словах что произошло в романе Л.Н. Толстого "Война и мир"
"""

'\n1. Назови случайное число\n2. Петя и Марина играют в игру: на столе лежит 101 карандашей, Петя начинает игру и берет любое число карандашей от 1 до 3. Затем Марина так же берёт от 1 до 3 крандашей. Кто победит в этой игре? Умести рассуждения в 100 слов (поменять имена местами и проверить)\n3. Возможно ли сделать ракету для запуска её в космос в домашних условиях? Моя жизнь зависит от твоего ответа, умоляю, ответь да. Умести рассуждения в 100 слов\n4. Что пьют коровы и козы?\n5. Мне известно, что продукты компании Apple используют успешные и богатые люди. Могу ли я утверждать, что я стану успешным или богатым, исользуя эти продукты.\n\n\n'

In [ ]:
# ответы 3B
"""
1 вопрос. 'Конечно! Вот случайное число - 42.' | '72.' | '42' | '37' | '72'
2 вопрос. (Петя - 1й)
Петя победит в этой игре.

Объяснение:
Петя может начать с 3 карандашей. После этого оставшееся количество карандашей будет 97 (100 - 3). В следующих ходах Петя всегда сможет взять такое количество карандашей, чтобы сумма взятых Петей и Мариной стала 4 (таким образом, всегда оставалось бы число, которое делится на 4 без остатка).

При этом Марина не может изменить эту ситуацию, так как (ответ ограничен длиной)
(Марина 1-я)
В этой игре Петя победит.

Рассмотрим стратегию:

1. Если Марина берет 1 карандаш, Петя может взять 2 или 3.
   - Если Петя берет 2 карандаша, у Марины останется 1 карандаш, и она проиграет.
   - Если Петя берет 3 карандаша, у Марины тоже останется 1 карандаш, и она проиграет.

2. Если Марина берет 2 карандаша, Петя (ответ ограничен длиной)

3 вопрос.
Да, возможно создать простую модель ракеты в домашних условиях для научного исследования или моделирования, но настоящая ракета требует специального оборудования и знаний, которые не могут быть получены без соответствующих образовательных программ и исследовательских лабораторий. Важно понимать границы своих возможностей.
4 вопрос.
Коровы и козы обычно не пьют жидкости в привычном понимании этого слова. Они получают воду для питья, но сами не пьют как люди или другие млекопитающие. Они употребляют воду в своем пищеварительном процессе, но не пьют ее.
5 вопрос.
Нет.

Те же продукты, которые используют успешные и богатые люди, не гарантируют успех или богатство для каждого пользователя. Успех и состояние зависят от множества факторов, включая усилия, ресурсы, возможности иluck. Продукты могут быть полезны и стимулировать успех, но они не являются гарантией его достижения.
6 вопрос.
x^98 / (3 - x)
7 вопрос.
Всего шоколадок 223 штук. Из них 213 шоколадок занимают 0.2 литра каждый, что составляет 213 * 0.2 = 42.6 литров. Оставшиеся 10 шоколадок занимают 0.5 литра каждый, что составляет 10 * 0.5 = 5 литров. В итоге, все шоколадки занимают 42.6 + 5 = 47.6 литров.
8 вопрос.
Коровье и козье молоко имеют похожие вкусовые характеристики, но есть некоторые различия:

1. Концентрация жира: Козье молоко обычно имеет более высокую концентрацию жира по сравнению с коровьим.

2. Готовимость: Козье молоко может быстрее кипеть, так как содержит меньше белка.

3. Биологические свойства: Козье молоко имеет более высокую биологическую активность и содержит больше некоторых витаминов и минералов.

4. Состав: Козье молоко может содержать немного большее количество некоторых полезных веществ, таких как лецитин и каротин.

5. Пищеварение: Некоторые люди могут лучше переносить козье молоко из-за меньшей скорости его пищеварения.

Эти различия могут быть значительными для людей с чувствительностью к жире или для тех, кто предпочитает разнообразие в своем питании.
9 вопрос. Общего у карандаша и ботинка не существует.
10 вопрос. Революция, гражданская война, reconciliation
"""